In [1]:
import os
import json
import pandas as pd
import numpy as np
import scipy as sp
import copy

In [8]:
playlists = list()
tracks = dict()
map_pl = list()

max_files_for_quick_processing = 5


def process_track(track):
    key = track['track_uri']
    if not key in tracks:
        tk = dict()
        tk['artist_name'] = track['artist_name']
        tk['artist_uri'] = track['artist_uri']
        tk['track_name'] = track['track_name']
        tk['album_uri'] = track['album_uri']
        tk['duration_ms'] = track['duration_ms']
        tk['album_name'] = track['album_name']
        tracks[track['track_uri']] = tk
    return key


def process_playlist(playlist):
    pl = dict()
    pl['name'] = playlist['name']
    pl['collaborative'] = playlist['collaborative']
    pl['pid'] = playlist['pid']
    pl['modified_at'] = playlist['modified_at']
    pl['num_albums'] = playlist['num_albums']
    pl['num_tracks'] = playlist['num_tracks']
    pl['num_followers'] = playlist['num_followers']
    pl['num_edits'] = playlist['num_edits']
    pl['duration_ms'] = playlist['duration_ms']
    pl['num_artists'] = playlist['num_artists']
    if 'description' in playlist:
        pl['description'] = playlist['description']
    else:
        pl['description'] = None
    trks = list()
    for track in playlist['tracks']:
        map_pl.append([playlist['pid'], track['track_uri']])
        trks.append(track['track_uri'])
        process_track(track)
    return pl

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        print(filename)
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            slice = json.loads(js)
            for playlist in slice['playlists']:
                playlists.append(process_playlist(playlist))
            count += 1
            if quick and count > max_files_for_quick_processing:
                break

quick = True
process_mpd('mpd.v1/data')

mpd.slice.0-999.json


In [9]:
print(len(playlists))
print(len(tracks))
print(len(map_pl))

1000
34443
67503


In [10]:
playlist_df = pd.DataFrame(playlists)
playlist_df.head()
print(playlist_df.describe())

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid
0,false,None,11532414,1493424000,Throwbacks,47,37,6,1,52,0
1,false,None,11656470,1506556800,Awesome Playlist,23,21,5,1,39,1
2,false,None,14039958,1505692800,korean,51,31,18,1,64,2
3,false,None,28926058,1501027200,mat,107,86,4,1,126,3
4,false,None,4335282,1401667200,90s,16,16,7,2,17,4


In [11]:
tracks_df = pd.DataFrame.from_dict(tracks, orient='index')
tracks_df.head()
print(tracks_df.describe())

,artist_name,artist_uri,track_name,album_uri,duration_ms,album_name
spotify:track:000mA0etY38nKdvf1N04af,The Coronas,spotify:artist:2tppd6KkhK4ULAd217Ecq1,If I Gave Myself To Someone Else,spotify:album:662PiU3dRsilN0Gp87IiSF,214506,The Long Way
spotify:track:000xQL6tZNLJzIrtIgxqSl,ZAYN,spotify:artist:5ZsFI1h6hIdQRw2ti0hz81,Still Got Time,spotify:album:2kGUeTGnkLOYlinKRJe47G,188490,Still Got Time
spotify:track:006AVH7fq061voGXkUiII4,Ryan O'Shaughnessy,spotify:artist:5xxkw4ElO0WX3v4i1W5kAQ,Fingertips,spotify:album:3MksslTJffj1jqxGkQt8Tm,191572,Fingertips
spotify:track:006PJvsr6CyV3JdBf7wiNF,Paul Wall,spotify:artist:0k7Xl1pqI3tu8sSEjo5oEg,Crumble the Satellite,spotify:album:0vUjMmaQwKyplM1cf5HQ8G,305320,slab god
spotify:track:006yrnQMCZpiUgkR612gC8,Desiigner,spotify:artist:7pFeBzX627ff0VnN6bxPR4,Caliber,spotify:album:0cHT4ll3sEPyFFWoFuibMl,123411,New English


In [12]:
playlist_map_df = pd.DataFrame(map_pl, columns=['pid', 'track_uri'])
playlist_map_df.head()
print(playlist_map_df.describe())

,pid,track_uri
0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,0,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,0,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,0,spotify:track:1lzr43nnXAijIGYnCT8M8H


In [61]:
merged = pd.merge(pd.merge(tracks_df, playlist_map_df, left_index=True, right_on='track_uri'), playlist_df, on='pid')

(67503, 19)


,artist_name,artist_uri,track_name,album_uri,duration_ms_x,album_name,pid,track_uri,collaborative,description,duration_ms_y,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,match
0,The Coronas,spotify:artist:2tppd6KkhK4ULAd217Ecq1,If I Gave Myself To Someone Else,spotify:album:662PiU3dRsilN0Gp87IiSF,214506,The Long Way,371,spotify:track:000mA0etY38nKdvf1N04af,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
1,Lewis Watson,spotify:artist:40ELTAg7Kg6vbWnlyx2n9R,stones around the sun,spotify:album:4cKBAg2zgjrVF2XefrW4WC,224440,the morning,371,spotify:track:0JGbwcwPV0VfuR4zDcZ9ce,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
2,Jaymes Young,spotify:artist:6QrQ7OrISRYIfS5mtacaw2,We Won't,spotify:album:6MuWCR3WPjwyKhqsTKLZ3z,240586,Feel Something,371,spotify:track:0Zge2Kfo3Yd9JOGnAmVPbb,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
3,John Lucas,spotify:artist:7iEy8zKFtlYIINaxxLIyBk,This Will Be Our Home,spotify:album:2N7sEVVS3jKMJJuJi4v0UF,277160,Promised Land,371,spotify:track:1Sw7fhf7YJCD6GcWW0wETD,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
4,Hozier,spotify:artist:2FXC3k01G6Gw61bmprjgqS,Cherry Wine - Live,spotify:album:36k5aXpxffjVGcNce12GLZ,240147,Hozier,371,spotify:track:1ivHxaGL5ld9VS1zsYc4YN,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
5,The Head and the Heart,spotify:artist:0n94vC3S9c3mb2HyNAOcjg,Rivers And Roads,spotify:album:4JKVVz1tD8PYKoXIZ7ZEJy,284000,The Head And The Heart,371,spotify:track:1t2tKmSYA61IA7scT1yoIn,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
6,Lewis Watson,spotify:artist:40ELTAg7Kg6vbWnlyx2n9R,forever (acoustic version),spotify:album:2T5RZubmhl8kwYxENFQkan,188040,midnight (acoustic),371,spotify:track:2AywFPR5CzTM5TNwEkzzxL,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
7,Ben Howard,spotify:artist:5schNIzWdI9gJ1QRK8SBnc,Only Love,spotify:album:2MxcbOGFi99D9JACvj74AH,249173,Every Kingdom,371,spotify:track:3MdYFBIzPf7lSJnI8wi3Ka,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
8,Ben Rector,spotify:artist:4AapPt7H6bGH4i7chTulpI,Hide Away,spotify:album:4ASegWfBk0kc5psPGAEcfE,199836,Something Like This,371,spotify:track:441XbO0qkxAEoPe1umyzkg,false,None,3675883,1496793600,quiet,15,14,4,1,15,1
9,Thirdstory,spotify:artist:7GJbWH8vhhuW22707B8HsW,Searching For A Feeling,spotify:album:7pjWbCevZabqDKwm26ggKd,230466,Searching,371,spotify:track:4VqJipc8kIFseGlVNKlv0W,false,None,3675883,1496793600,quiet,15,14,4,1,15,1


In [ ]:
#Match column indicated whether a song was truly in a playlist
merged["match"] = 1
print(merged.shape)
merged.head(20)

In [ ]:
#Build a new DataFrame in which each playlist is filled with random songs.  The number of songs for each
#playlist remains the same.

negative_training = copy.deepcopy(merged)
negative_training["match"] = 0   #Because these songs are not actually in the playlists

#Take songs from random rows in merged df
random_rows = np.random.randint(len(negative_training), size = len(negative_training))

#replace these columns with those from a random row
replace = ["artist_name", "artist_uri", "track_name", "album_uri", "duration_ms_x", "album_name", "track_uri"]
for index in range(len(negative_training)):
    for column in replace:
        negative_training.loc[index, [column]] = negative_training.loc[random_rows[index], [column]]

#Fix other columns to correctly identify unique albums etc...
for pid, df in negative_training.groupby(["pid"]):   
    unique_albums = df["album_name"].nunique()
    num_artists = df["artist_name"].nunique()
    duration_ms_y = np.sum(df["duration_ms_x"])
    
    rows = df.index.values
    for row in rows:
        negative_training.loc[row, "num_albums"] = unique_albums
        negative_training.loc[row, "num_artists"] = num_artists
        negative_training.loc[row, "duration_ms_y"] = duration_ms_y

negative_training.head(20)
    